# 📊 Sales Dataset EDA - Phân Tích Dữ Liệu Bán Hàng

## 🎯 Mục Tiêu
Phân tích khám phá dữ liệu bán hàng để hiểu:
- Xu hướng bán hàng theo thời gian
- Phân tích theo sản phẩm, khu vực, khách hàng
- Seasonal patterns và trends
- Customer behavior analysis

## 📋 Dataset Overview
- **Nguồn**: Synthetic Sales Data
- **Thời gian**: 2 năm (2022-2023)
- **Features**: Date, Product, Category, Region, Customer, Sales, Quantity, Price
- **Mục tiêu**: Time series analysis, seasonal patterns, customer segmentation

## 🔍 Kỹ Thuật Sẽ Sử Dụng
- Time series analysis
- Seasonal decomposition
- Customer segmentation
- Product performance analysis
- Geographic analysis
- Trend analysis


In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Cài đặt style cho plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Đã import thành công tất cả thư viện!")
print("📊 Sẵn sàng bắt đầu phân tích Sales Dataset!")


## 📊 Bước 1: Tạo Synthetic Sales Dataset

Tạo dataset bán hàng tổng hợp với các đặc điểm thực tế:
- Dữ liệu 2 năm (2022-2023)
- 5 sản phẩm chính với 3 categories
- 4 khu vực bán hàng
- 1000+ khách hàng
- Seasonal patterns và trends


In [ ]:
# Tạo synthetic sales dataset
np.random.seed(42)

# Thông tin cơ bản
start_date = '2022-01-01'
end_date = '2023-12-31'
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Sản phẩm và categories
products = {
    'Laptop Gaming': 'Electronics',
    'iPhone 14': 'Electronics', 
    'Nike Air Max': 'Fashion',
    'Adidas Ultraboost': 'Fashion',
    'MacBook Pro': 'Electronics',
    'Samsung Galaxy': 'Electronics',
    'Levi\'s Jeans': 'Fashion',
    'Zara Jacket': 'Fashion',
    'iPad Air': 'Electronics',
    'Nike T-Shirt': 'Fashion'
}

# Khu vực
regions = ['North', 'South', 'East', 'West']

# Tạo dữ liệu
sales_data = []

for date in date_range:
    # Số lượng giao dịch trong ngày (có seasonal pattern)
    day_of_year = date.timetuple().tm_yday
    seasonal_factor = 1 + 0.3 * np.sin(2 * np.pi * day_of_year / 365)  # Seasonal pattern
    weekend_factor = 0.7 if date.weekday() >= 5 else 1.0  # Weekend effect
    holiday_factor = 1.5 if date.month in [11, 12] else 1.0  # Holiday season
    
    num_transactions = int(np.random.poisson(50 * seasonal_factor * weekend_factor * holiday_factor))
    
    for _ in range(num_transactions):
        product = np.random.choice(list(products.keys()))
        category = products[product]
        region = np.random.choice(regions)
        customer_id = f"CUST_{np.random.randint(1000, 9999)}"
        
        # Giá sản phẩm (có variation theo thời gian)
        base_prices = {
            'Laptop Gaming': 1200, 'iPhone 14': 800, 'Nike Air Max': 120,
            'Adidas Ultraboost': 150, 'MacBook Pro': 2000, 'Samsung Galaxy': 600,
            'Levi\'s Jeans': 80, 'Zara Jacket': 120, 'iPad Air': 500, 'Nike T-Shirt': 30
        }
        
        base_price = base_prices[product]
        price_variation = np.random.normal(1, 0.1)  # 10% variation
        price = base_price * price_variation
        
        # Số lượng (thường 1-3 items)
        quantity = np.random.choice([1, 2, 3], p=[0.7, 0.2, 0.1])
        
        # Total sales
        sales = price * quantity
        
        # Discount factor (10% chance of discount)
        if np.random.random() < 0.1:
            discount = np.random.uniform(0.05, 0.25)
            sales *= (1 - discount)
        
        sales_data.append({
            'Date': date,
            'Product': product,
            'Category': category,
            'Region': region,
            'Customer_ID': customer_id,
            'Price': round(price, 2),
            'Quantity': quantity,
            'Sales': round(sales, 2),
            'Year': date.year,
            'Month': date.month,
            'Day': date.day,
            'Weekday': date.weekday(),
            'Quarter': date.quarter
        })

# Tạo DataFrame
df = pd.DataFrame(sales_data)

print(f"✅ Đã tạo thành công Sales Dataset!")
print(f"📊 Kích thước dataset: {df.shape}")
print(f"📅 Thời gian: {df['Date'].min()} đến {df['Date'].max()}")
print(f"💰 Tổng doanh thu: ${df['Sales'].sum():,.2f}")
print(f"🛍️ Tổng số giao dịch: {len(df):,}")


## 📊 Bước 2: Data Loading & Overview


In [ ]:
# Kiểm tra thông tin cơ bản về dataset
print("🔍 THÔNG TIN CƠ BẢN VỀ DATASET")
print("=" * 50)
print(f"📊 Shape: {df.shape}")
print(f"💾 Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"📅 Date range: {df['Date'].min()} đến {df['Date'].max()}")
print(f"📈 Total days: {(df['Date'].max() - df['Date'].min()).days + 1}")

print("\n📋 COLUMNS INFO:")
print("=" * 30)
print(df.info())

print("\n🔢 DATA TYPES:")
print("=" * 20)
print(df.dtypes)

print("\n📊 SAMPLE DATA (5 rows đầu):")
print("=" * 35)
df.head()


In [ ]:
# Kiểm tra missing values
print("🔍 MISSING VALUES ANALYSIS")
print("=" * 40)
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent
})

print(missing_df[missing_df['Missing Count'] > 0])

if missing_df['Missing Count'].sum() == 0:
    print("✅ Không có missing values trong dataset!")

# Kiểm tra duplicate rows
duplicates = df.duplicated().sum()
print(f"\n🔄 Duplicate rows: {duplicates}")

# Kiểm tra unique values cho categorical columns
print("\n📊 UNIQUE VALUES:")
print("=" * 25)
categorical_cols = ['Product', 'Category', 'Region', 'Customer_ID']
for col in categorical_cols:
    unique_count = df[col].nunique()
    print(f"{col}: {unique_count} unique values")
    
    if col in ['Product', 'Category', 'Region']:
        print(f"  Values: {list(df[col].unique())}")
    print()


## 📊 Bước 3: Data Profiling


In [ ]:
# Statistical summary cho numerical columns
print("📊 STATISTICAL SUMMARY")
print("=" * 30)
numerical_cols = ['Price', 'Quantity', 'Sales']
print(df[numerical_cols].describe().round(2))

# Thêm một số thống kê bổ sung
print("\n📈 ADDITIONAL STATISTICS")
print("=" * 35)
for col in numerical_cols:
    print(f"\n{col}:")
    print(f"  Mean: ${df[col].mean():.2f}")
    print(f"  Median: ${df[col].median():.2f}")
    print(f"  Std: ${df[col].std():.2f}")
    print(f"  Min: ${df[col].min():.2f}")
    print(f"  Max: ${df[col].max():.2f}")
    print(f"  Skewness: {df[col].skew():.3f}")
    print(f"  Kurtosis: {df[col].kurtosis():.3f}")


In [ ]:
# Phân tích categorical features
print("📊 CATEGORICAL FEATURES ANALYSIS")
print("=" * 40)

# Product analysis
print("\n🛍️ PRODUCT ANALYSIS:")
print("-" * 25)
product_stats = df.groupby('Product').agg({
    'Sales': ['count', 'sum', 'mean'],
    'Quantity': 'sum',
    'Price': 'mean'
}).round(2)
product_stats.columns = ['Transactions', 'Total_Sales', 'Avg_Sales', 'Total_Quantity', 'Avg_Price']
product_stats = product_stats.sort_values('Total_Sales', ascending=False)
print(product_stats)

# Category analysis
print("\n📂 CATEGORY ANALYSIS:")
print("-" * 25)
category_stats = df.groupby('Category').agg({
    'Sales': ['count', 'sum', 'mean'],
    'Quantity': 'sum',
    'Price': 'mean'
}).round(2)
category_stats.columns = ['Transactions', 'Total_Sales', 'Avg_Sales', 'Total_Quantity', 'Avg_Price']
category_stats = category_stats.sort_values('Total_Sales', ascending=False)
print(category_stats)

# Region analysis
print("\n🌍 REGION ANALYSIS:")
print("-" * 20)
region_stats = df.groupby('Region').agg({
    'Sales': ['count', 'sum', 'mean'],
    'Quantity': 'sum',
    'Price': 'mean'
}).round(2)
region_stats.columns = ['Transactions', 'Total_Sales', 'Avg_Sales', 'Total_Quantity', 'Avg_Price']
region_stats = region_stats.sort_values('Total_Sales', ascending=False)
print(region_stats)


## 📊 Bước 4: Missing Value Analysis

Vì dataset này là synthetic data nên không có missing values, nhưng chúng ta sẽ kiểm tra và tạo visualization để hiểu pattern.


In [ ]:
# Missing value visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Missing value heatmap
missing_data = df.isnull().sum()
if missing_data.sum() > 0:
    sns.heatmap(df.isnull(), yticklabels=False, cbar=True, cmap='viridis', ax=axes[0])
    axes[0].set_title('Missing Values Heatmap')
else:
    axes[0].text(0.5, 0.5, '✅ No Missing Values', ha='center', va='center', 
                transform=axes[0].transAxes, fontsize=16, color='green')
    axes[0].set_title('Missing Values Status')

# Data completeness
completeness = (1 - missing_data / len(df)) * 100
bars = axes[1].bar(range(len(completeness)), completeness.values, color='lightgreen')
axes[1].set_title('Data Completeness by Column')
axes[1].set_xlabel('Columns')
axes[1].set_ylabel('Completeness (%)')
axes[1].set_xticks(range(len(completeness)))
axes[1].set_xticklabels(completeness.index, rotation=45)
axes[1].set_ylim(0, 105)

# Add percentage labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height + 1,
                f'{height:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("✅ Dataset hoàn toàn clean - không có missing values!")
